In [7]:
import json
import os
import pandas as pd
import numpy as np
from collections import Counter

# 檔案路徑配置
files = {
    "Option 1 (Baseline) - Case 1": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_1\json\processed\op1_100_case_1.json",
    "Option 1 (Baseline) - Case 2": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_1\json\processed\op1_100_case_2.json",
    "Option 2 (Context) - Case 1": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_2\json\processed\op2_100_case_1.json",
    "Option 2 (Context) - Case 2": r"C:\Users\bmwlab\Desktop\Testing_Row_Data\option_2\json\processed\op2_100_case_2.json"
}

In [8]:
def calculate_hhi(items_list):
    """
    計算赫芬達爾—赫希曼指數 (HHI)。
    用於衡量資料的集中程度。範圍從 1/N 到 1.0。
    """
    if not items_list:
        return 0.0
    
    total = len(items_list)
    counts = Counter(items_list)
    hhi = sum((count / total) ** 2 for count in counts.values())
    return hhi

def analyze_file(name, filepath):
    """讀取 JSON 並計算各項指標"""
    if not os.path.exists(filepath):
        print(f"⚠️ Warning: File not found: {filepath}")
        return None

    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"❌ Error reading {filepath}: {e}")
        return None

    modules = []
    keys = []
    hierarchy_depths = []
    key_lengths = []

    for item in data:
        # 1. 模組提取
        mod = item.get('affected_module', 'Unknown')
        modules.append(mod)

        # 2. 配置鍵提取與深度分析
        key = item.get('modified_key', '')
        keys.append(key)
        
        if key:
            # 階層深度：以點號分隔計算數量 (e.g., 'a.b.c' -> 3)
            h_depth = len(key.split('.'))
            hierarchy_depths.append(h_depth)
            key_lengths.append(len(key))
        else:
            hierarchy_depths.append(0)
            key_lengths.append(0)

    # 數據計算
    module_hhi = calculate_hhi(modules)
    key_hhi = calculate_hhi(keys)
    avg_h_depth = np.mean(hierarchy_depths) if hierarchy_depths else 0
    avg_k_len = np.mean(key_lengths) if key_lengths else 0
    
    # 統計出現次數最多的模組
    top_mod_count = Counter(modules).most_common(1)
    top_module_str = f"{top_mod_count[0][0]} ({top_mod_count[0][1]})" if top_mod_count else "None"

    return {
        "Name": name,
        "Total Cases": len(data),
        "Module HHI": module_hhi,
        "Key HHI": key_hhi,
        "Avg Hierarchy Level": avg_h_depth,
        "Avg Key Length": avg_k_len,
        "Top Module": top_module_str
    }

In [9]:
results = []
for name, path in files.items():
    res = analyze_file(name, path)
    if res:
        results.append(res)

if results:
    df = pd.DataFrame(results)
    
    # 數值格式化處理
    df["Module HHI"] = df["Module HHI"].map("{:.4f}".format)
    df["Key HHI"] = df["Key HHI"].map("{:.4f}".format)
    df["Avg Hierarchy Level"] = df["Avg Hierarchy Level"].map("{:.2f}".format)
    df["Avg Key Length"] = df["Avg Key Length"].map("{:.2f}".format)
    
    # 重新排序欄位以利閱讀
    cols = ["Name", "Total Cases", "Module HHI", "Key HHI", "Avg Hierarchy Level", "Avg Key Length", "Top Module"]
    df = df[cols]
    
    print("### Fault Concentration & Modification Depth Analysis Results ###")
    display(df) # 在 Notebook 中顯示漂亮表格
    
    print("\n💡 指標解讀說明:")
    print("- HHI 越高：代表錯誤越集中。")
    print("- Module HHI：衡量錯誤是否集中在特定軟體模組 (如 PHY)。")
    print("- Key HHI：衡量是否重複修改特定的配置參數。")
    print("- Avg Hierarchy Level：數值越高，代表修改的參數在配置樹中越深。")
else:
    print("沒有找到可分析的資料。")

### Fault Concentration & Modification Depth Analysis Results ###


,Name,Total Cases,Module HHI,Key HHI,Avg Hierarchy Level,Avg Key Length,Top Module
0,Option 1 (Baseline) - Case 1,100,0.1998,0.0118,2.46,34.46,RRC (36)
1,Option 1 (Baseline) - Case 2,100,0.2316,0.0132,2.53,35.26,PHY (39)
2,Option 2 (Context) - Case 1,100,0.3034,0.0160,2.78,48.61,PHY (40)
3,Option 2 (Context) - Case 2,100,0.4038,0.0118,2.62,39.56,PHY (60)



💡 指標解讀說明:
- HHI 越高：代表錯誤越集中。
- Module HHI：衡量錯誤是否集中在特定軟體模組 (如 PHY)。
- Key HHI：衡量是否重複修改特定的配置參數。
- Avg Hierarchy Level：數值越高，代表修改的參數在配置樹中越深。
